In [14]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import pandas as pd

# ----- CONFIGURATION -----
BASE_PATH = r"E:\Fabric Defect Detection\fabric_dataset"
CSV_PATH = os.path.join(BASE_PATH, "E:\\Fabric Defect Detection\\fabric_types.csv")
IMG_SIZE = 128
BATCH_SIZE = 4
EPOCHS = 30
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----- LOAD FABRIC TYPES -----
fabric_df = pd.read_csv(CSV_PATH)

# Get unique fabric types and assign class IDs
unique_fabrics = sorted(fabric_df["Fabric_Type"].unique())
name_to_class_id = {name: idx for idx, name in enumerate(unique_fabrics)}
class_id_to_name = {idx: name for name, idx in name_to_class_id.items()}

# ----- TRANSFORMS -----
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

# ----- CUSTOM DATASET -----
class FabricDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.df = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['Image'].strip('"')  # <-- fix here
        fabric_type = self.df.iloc[idx]['Fabric_Type']
        label = name_to_class_id[fabric_type]

        image = Image.open(img_path).convert("RGB")
        if self.transform:
           image = self.transform(image)
        return image, label


# ----- DATALOADERS -----
dataset = FabricDataset(CSV_PATH, transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)

# ----- CNN MODEL -----
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * (IMG_SIZE // 8) * (IMG_SIZE // 8), 128), nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)

num_classes = len(class_id_to_name)
model = SimpleCNN(num_classes).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ----- TRAINING LOOP -----
def train_model():
    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss:.4f}")

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        print(f"Validation Accuracy: {100 * correct / total:.2f}%")

train_model()

# ----- TESTING ON NEW IMAGE -----
def predict_image(image_path):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        predicted_class = class_id_to_name[predicted.item()]
        print(f"Predicted fabric type: {predicted_class}")

# Example usage:
predict_image(r"dataset\\non_defect\\02.jpg")


Epoch 1/30, Loss: 14.6182
Validation Accuracy: 12.50%
Epoch 2/30, Loss: 13.2722
Validation Accuracy: 12.50%
Epoch 3/30, Loss: 13.8094
Validation Accuracy: 12.50%
Epoch 4/30, Loss: 13.2057
Validation Accuracy: 25.00%
Epoch 5/30, Loss: 11.6285
Validation Accuracy: 25.00%
Epoch 6/30, Loss: 11.7136
Validation Accuracy: 12.50%
Epoch 7/30, Loss: 10.9430
Validation Accuracy: 25.00%
Epoch 8/30, Loss: 10.4945
Validation Accuracy: 12.50%
Epoch 9/30, Loss: 11.2057
Validation Accuracy: 37.50%
Epoch 10/30, Loss: 9.6789
Validation Accuracy: 12.50%
Epoch 11/30, Loss: 10.5349
Validation Accuracy: 25.00%
Epoch 12/30, Loss: 8.8777
Validation Accuracy: 25.00%
Epoch 13/30, Loss: 10.0457
Validation Accuracy: 37.50%
Epoch 14/30, Loss: 7.2549
Validation Accuracy: 12.50%
Epoch 15/30, Loss: 7.4115
Validation Accuracy: 50.00%
Epoch 16/30, Loss: 6.4635
Validation Accuracy: 12.50%
Epoch 17/30, Loss: 8.1661
Validation Accuracy: 25.00%
Epoch 18/30, Loss: 7.4486
Validation Accuracy: 12.50%
Epoch 19/30, Loss: 6.4111
